In [1]:
%load_ext autoreload
%autoreload 2

In [103]:
import hvplot
import hvplot.xarray
import numpy as np
import xarray

import gnss_tid.parameter

In [ ]:
file = "outputs/2024-12-06/22-20-41/autofocus.h5"
data = xarray.open_dataset(file)

BLOCK_SIZE = 32
STEP_SIZE = 8
NFFT = 128

plots = gnss_tid.parameter.estimate_parameters_block_debug(data, NFFT, BLOCK_SIZE, STEP_SIZE)
hvplot.show(plots)

Launching server at http://localhost:37927


In [102]:
import holoviews as hv
hv.help(hv.VLine)

VLine

Online example: https://holoviews.org/reference/elements/bokeh/VLine.html

-------------
Style Options
-------------

	alpha, color, hover_alpha, hover_color, hover_line_alpha, hover_line_cap, hover_line_color, hover_line_dash, hover_line_dash_offset, hover_line_join, hover_line_width, level, line_alpha, line_cap, line_color, line_dash, line_dash_offset, line_join, line_width, muted_alpha, muted_color, muted_line_alpha, muted_line_cap, muted_line_color, muted_line_dash, muted_line_dash_offset, muted_line_join, muted_line_width, nonselection_alpha, nonselection_color, nonselection_line_alpha, nonselection_line_cap, nonselection_line_color, nonselection_line_dash, nonselection_line_dash_offset, nonselection_line_join, nonselection_line_width, selection_alpha, selection_color, selection_line_alpha, selection_line_cap, selection_line_color, selection_line_dash, selection_line_dash_offset, selection_line_join, selection_line_width, visible

(Consult bokeh's documentation for more inf

In [ ]:
import colorcet as cc

P = params["coherence"] * params["R"]

p1 = data["image"].hvplot.image(x="x", y="y", data_aspect=1, cmap=cc.cm.diverging_bwr_55_98_c37, clim=(-.3, .3), groupby="time")
# p2 = params["R"].hvplot.contour(x="px", y="py", data_aspect=1, groupby="time", levels=[500, 750, 1000, 2000])
# p2 = params["R"].hvplot.image(x="px", y="py", data_aspect=1, groupby="time", cmap=cc.cm.linear_grey_0_100_c0, alpha=.2, clim=(500, 2000))
p2 = params["coherence"].hvplot.image(x="px", y="py", data_aspect=1, groupby="time", cmap=cc.cm.linear_grey_0_100_c0, alpha=.2, clim=(.9, .95))
# p2 = params["coherence"].hvplot.contour(x="px", y="py", data_aspect=1, groupby="time", levels=[.9, .925, .95])
# p2 = P.hvplot.image(x="px", y="py", data_aspect=1, groupby="time", cmap=cc.cm.linear_grey_0_100_c0, alpha=.2)
hvplot.show(p1 * p2 + p2)

Launching server at http://localhost:44487


In [111]:
data

<xarray.Dataset> Size: 129MB
Dimensions:     (time: 109, y: 148, x: 224, py: 8, px: 12, ky: 32, kx: 32, ci: 2)
Coordinates:
  * x           (x) float64 2kB -2.231e+03 -2.211e+03 ... 2.204e+03 2.224e+03
  * y           (y) float64 1kB -1.318e+03 -1.299e+03 ... 1.59e+03 1.61e+03
  * time        (time) datetime64[ns] 872B 2015-03-25T22:00:00 ... 2015-03-25...
  * kx          (kx) float64 256B 0.0 0.001563 0.003125 ... -0.003125 -0.001563
  * ky          (ky) float64 256B 0.0 0.001563 0.003125 ... -0.003125 -0.001563
  * px          (px) float64 96B -1.911e+03 -1.592e+03 ... 1.285e+03 1.605e+03
  * py          (py) float64 64B -999.7 -680.9 -362.1 ... 594.2 913.0 1.232e+03
Dimensions without coordinates: ci
Data variables:
    image       (time, y, x) float64 29MB 0.02883 0.03004 ... 0.01397 0.0149
    density     (time, y, x) int32 14MB 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    patch       (time, py, px, ky, kx) float64 86MB ...
    Fx          (time, py, px) float64 84kB ...
    Fy          (time, py, px) float64 84kB ...
    F           (time, py, px) float64 84kB ...
    height      (time) int32 436B ...
    objective   (time) float64 872B ...
    wavelength  (time) float64 872B ...
    offset      (time) float64 872B ...
    phase       (time) float32 436B ...
    center      (ci) float64 16B ...

In [ ]:
p1 = data["image"].hvplot.image(x="x", y="y", data_aspect=1, cmap=cc.cm.diverging_bwr_55_98_c37, clim=(-.3, .3), groupby="time", frame_width=1000)

V = params["vx"] + params["vy"] * 1j
va = xarray.ufuncs.angle(V) + np.pi
vm = abs(V)
V = xarray.Dataset({"mag": vm, "ang": va, "coherence": params["coherence"]})
p2 = V.where(V.coherence > .93).hvplot.vectorfield(x="px", y="py", angle="ang", mag="mag", groupby="time", data_aspect=1).opts(magnitude='mag')

p3 = data["density"].hvplot.image(x="x", y="y", data_aspect=1, groupby="time", clim=(0, 50))
hvplot.show((p1 * p2 + p3).cols(1))

Launching server at http://localhost:35765
